In [1]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

# Function to compute and save spectrogram for a single audio file
def compute_and_save_spectrogram(file_path, output_dir, show_plot=False, hop=128):
    # Load audio file
    y, sr = librosa.load(file_path, sr=None)
    
    #win_sizes = [1028,2048,4096,5632,8192]
    win_sizes = [4096]
    for win in win_sizes:
        # Compute STFT spectrogram
        D = np.abs(librosa.stft(y, n_fft=win, window="hann", hop_length=hop, win_length=win))
        
        # Plot spectrogram if show_plot is True
        if show_plot:
            #plt.figure(figsize=(14, 10), dpi=100)
            librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), hop_length=hop, 
                                    y_axis='log', x_axis='time', sr=sr)
            plt.colorbar(format='%+2.0f dB')
            plt.title('Log-frequency power spectrogram')
            
            # Customize x-axis ticks for more detailed time intervals
            duration = librosa.get_duration(y=y, sr=sr)
            plt.xticks(np.linspace(0, duration, num=25), np.round(np.linspace(0, duration, num=25), 1))
            plt.show()  # Display the plot
        
        # Save spectrogram as an image file
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir, file_name + f'_spectrogram_{win}_{hop}.png')
        plt.figure(figsize=(14,10), dpi=100)
        librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), hop_length=hop, 
                                y_axis='log', x_axis='time', sr=sr, n_fft=win)
        
        # Customize x-axis ticks for more detailed time intervals
        duration = librosa.get_duration(y=y, sr=sr)
        plt.xticks(np.linspace(0, duration, num=15), np.round(np.linspace(0, duration, num=15), 1))
        
        plt.colorbar(format='%+2.0f dB')
        input_dir, filename = os.path.split(file_path)
        plt.title(f'{filename}\nLog-frequency power spectrogram\nwindow size: {win}\nhop length: {hop}')
        plt.savefig(output_file, bbox_inches='tight', dpi=100)
        #plt.savefig(output_file, dpi=100)
        plt.close()

# Directory containing .wav files
input_dir = r'C:\SoundCheck 21\data\DATA\Initial measurements\Test'
input_dir2 = r'C:\SoundCheck 21\data\DATA\Initial measurements\Test\Normalized'

# Output directory to save spectrograms
output_dir = r'C:\SoundCheck 21\data\DATA\Initial measurements\Test\spectrograms'

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Process each .wav file in the input directory
i = 0
for file in os.listdir(input_dir):
    if file.endswith('.wav'):
        file_path = os.path.join(input_dir, file)
        #file_path = r"C:\SoundCheck 21\data\DATA\Initial measurements\Test measurements\3 noise Measurements.wav"
        print(file_path)
        #compute_and_save_spectrogram(file_path, output_dir, show_plot=False, hop=256)
        compute_and_save_spectrogram(file_path, output_dir, show_plot=False, hop=512)
        print("Spectrograms saved successfully.")
        
for file in os.listdir(input_dir2):
    if file.endswith('.wav'):
        file_path = os.path.join(input_dir2, file)
        print(file_path)
        compute_and_save_spectrogram(file_path, output_dir, show_plot=False, hop=512)
        print("Spectrograms saved successfully.")
        



C:\SoundCheck 21\data\DATA\Initial measurements\Test\FAST - bad driver Slow-Fast Sweep 16-04-2024 125647.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\FAST - good driver Slow-Fast Sweep 16-04-2024 123606.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\GAIN 5 FAST - bad driver Slow-Fast Sweep 16-04-2024 133915.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\GAIN 5 FAST - good driver Slow-Fast Sweep 16-04-2024 134751.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\GAIN 5 SLOW - bad driver Slow-Fast Sweep 16-04-2024 134023.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\GAIN 5 SLOW - good driver Slow-Fast Sweep 16-04-2024 134703.wav
Spectrograms saved successfully.
C:\SoundCheck 21\data\DATA\Initial measurements\Test\GAIN FAST - bad driver Slow-Fast Sweep 16-04-2024 133418.wav
Spe

In [ ]:
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt

def plot_waveform(audio_path, gain=1.0):
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Calculate the time array
    duration = len(y) / sr
    time = np.linspace(0, duration, len(y))

    # Plot the waveform
    plt.figure(figsize=(10, 4))
    plt.plot(time, y*gain, color='b')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.grid(True)
    plt.show()

# Example usage
audio_file_path = r"C:\SoundCheck 21\data\DATA\Initial measurements\Test 9 Slow-Fast Sweep 16-04-2024 114409.wav"
plot_waveform(audio_file_path, gain=1.0)


In [11]:
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
import os

def apply_log_gain(audio_path, target_db=None):
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Calculate the current RMS level in dB
    rms_db = librosa.amplitude_to_db(librosa.feature.rms(y=y))

    # Calculate the dB adjustment needed to reach the target level
    if target_db is not None:
        db_adjustment = target_db - np.mean(rms_db)
    else:
        db_adjustment = 0  # No adjustment if target_db is not specified

    # Apply gain adjustment in dB scale
    y_normalized = y * (10 ** (db_adjustment / 20))  # Adjust amplitude based on dB
    
    # Write out normalized audio as WAV
    # Extract original filename and directory from the input audio path
    input_dir, input_filename = os.path.split(audio_path)
    output_filename = f"Normalized Log - {input_filename}"
    output = os.path.join(input_dir, output_filename)
    #print(output)
    sf.write(output_filename, y_normalized, sr)


def apply_linear_gain(audio_path, target_db=None):
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Calculate the time array
    duration = len(y) / sr
    time = np.linspace(0, duration, len(y))

    # Normalize the audio to a target dB level if specified
    if target_db is not None:
        # Calculate the current RMS dB
        rms_db = librosa.amplitude_to_db(librosa.feature.rms(y=y))

        # Calculate the dB adjustment needed to reach the target level
        db_adjustment = target_db - np.mean(rms_db)

        # Apply gain adjustment in linear scale
        y_normalized = librosa.util.normalize(y) * (10 ** (db_adjustment / 20))
    else:
        # If no target_db specified, normalize to the maximum amplitude
        y_normalized = librosa.util.normalize(y)

    # Plot the waveform
    #plt.figure(figsize=(10, 4))
    #plt.plot(time, y_normalized, color='b')
    #plt.xlabel('Time (s)')
    #plt.ylabel('Amplitude')
    #plt.title('Normalized Waveform')
    #plt.grid(True)
    #plt.show()
    
    # Write out normalized audio as WAV
    # Extract original filename and directory from the input audio path
    input_dir, input_filename = os.path.split(audio_path)
    output_filename = f"Normalized Linear - {input_filename}"
    output = os.path.join(input_dir, output_filename)
    #print(output)
    sf.write(output_filename, y_normalized, sr)

# Example usage with normalization to a specific dB level
#input_dir = r"C:\SoundCheck 21\data\DATA\Initial measurements\Test"
input_dir = r"C:\SoundCheck 21\data\DATA\Initial measurements\17-04-24\waveforms"

files = os.listdir(input_dir)
for file in files:
    if file.endswith('.wav'):
        file_path = os.path.join(input_dir, file)
        print(file_path)
        #apply_log_gain(file_path, target_db = -3)  # Target RMS level in dB
        apply_linear_gain(file_path, target_db = -6)  # Target RMS level in dB

C:\SoundCheck 21\data\DATA\Initial measurements\17-04-24\waveforms\1 - Fast Sweep - Fast-Slow Sweep 17-04-2024 164252.wav
C:\SoundCheck 21\data\DATA\Initial measurements\17-04-24\waveforms\1 - Slow Sweep - Fast-Slow Sweep 17-04-2024 164532.wav
C:\SoundCheck 21\data\DATA\Initial measurements\17-04-24\waveforms\3 - Fast Sweep - Fast-Slow Sweep 17-04-2024 163626.wav
C:\SoundCheck 21\data\DATA\Initial measurements\17-04-24\waveforms\3 - Slow Sweep - Fast-Slow Sweep 17-04-2024 163736.wav
